# Baseline App Analysis: Calm User Reviews

**Milestone 3: Data Analysis (Part 2)**

This notebook serves as the "control group" for our main analysis. We will perform the exact same topic modeling pipeline on the user reviews for **Calm**, a popular but non-conversational wellness app.

**Objective:**
By identifying the primary complaint themes for a baseline app, we can compare them to our conversational app findings. This will allow us to isolate the failure modes that are **unique to conversational AI**.

 🧼 Import all necessary libraries UNCOMMENT AND RUN IT
 📦 Install all required packages quietly
 !pip install bertopic[visualization] --quiet
 !pip install hdbscan --quiet
 !pip install umap-learn --quiet

In [ ]:
import warnings
import pandas as pd
from bertopic import BERTopic 
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from torch.utils.data import DataLoader, TensorDataset
from tqdm.notebook import tqdm
import plotly.express as px
from IPython.display import display

warnings.filterwarnings("ignore")
print("✅ Setup complete. All libraries are ready.")

In [5]:
print ("Loading the baseline app dataset (Calm)...")


DATA_PATH = '/kaggle/input/user-reviews-of-mental-health-chatbots-2022-2025/baseline_app_dataset.csv'

try:
    df_base = pd.read_csv(DATA_PATH)

    df_base.dropna(subset = ["review_text"], inplace = True)
    df_base['review_text'] = df_base['review_text'].astype(str).str.lower().str.strip()
    df_base = df_base[df_base['review_text'].str.len() > 15]

    docs_base = df_base['review_text'].tolist()
    print(f"successfully loaded and cleaned {len(docs_base)} documents for modeling.")

except FileNotFoundError:
    print(f"Error: Dataset not found at {DATA_PATH}. Please upload the file and check the path.")
    

Loading the baseline app dataset (Calm)...
successfully loaded and cleaned 8506 documents for modeling.


In [6]:
# --- Step 3: Configure the Topic Model ---
print("Configuring a reproducible BERTopic model...")

# A. Define a Stopword List (Identical to the first analysis)
stop_words = [
    "app", "replika", "wysa", "woebot", "calm", "bot", "ai", "like", "feel", "good", 
    "great", "nice", "love", "best", "amazing", "awesome", "fun", "ok", "cool",
    "me", "it", "and", "to", "the", "my", "is", "of", "with", "that", "for", "you", "but",
    "so", "on", "was", "this", "have", "in", "be", "as", "at", "not", "just", "are",
    "get", "want", "use", "go", "know", "say", "see", "think", "really", "even", "also"
]

# B. Define Deterministic Components (Identical)
random_seed = 42
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=random_seed)
hdbscan_model = HDBSCAN(min_cluster_size=30, metric='euclidean', cluster_selection_method='eom', prediction_data=True) # min_cluster_size can be smaller for smaller dataset
vectorizer_model = CountVectorizer(stop_words=stop_words, ngram_range=(1, 2))

# C. Initialize the final BERTopic model (Identical)
topic_model_base = BERTopic(
    language="multilingual",
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    vectorizer_model=vectorizer_model,
    min_topic_size=30, # Adjusted for a potentially smaller dataset
    verbose=True
)

print("BERTopic model is configured and ready for training.")

Configuring a reproducible BERTopic model...
BERTopic model is configured and ready for training.


In [6]:
print(f"Training BERTopic on {len(docs_base)} basline documents. This will take a few minutes...")

topics_base, probs_base = topic_model_base.fit_transform(docs_base)

print("n\ Baseline model trainig complete")

2025-07-21 09:19:08,795 - BERTopic - Embedding - Transforming documents to embeddings.


Training BERTopic on 8506 basline documents. This will take a few minutes...


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/266 [00:00<?, ?it/s]

2025-07-21 09:19:28,234 - BERTopic - Embedding - Completed ✓
2025-07-21 09:19:28,235 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-07-21 09:20:08,003 - BERTopic - Dimensionality - Completed ✓
2025-07-21 09:20:08,005 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-07-21 09:20:08,382 - BERTopic - Cluster - Completed ✓
2025-07-21 09:20:08,390 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-07-21 09:20:08,950 - BERTopic - Representation - Completed ✓


n\ Baseline model trainig complete


In [7]:
# --- Step 5: Review the Discovered Topics ---
print("Displaying the baseline topic overview...")

topic_info_base = topic_model_base.get_topic_info()
display(topic_info_base)

print("\n--- Detailed View of Top 10 Baseline Topics ---")
for topic_id in range(10): 
    if topic_id in topic_model_base.get_topics():
        print(f"\n--- Words for Baseline Topic #{topic_id} ---")
        print(topic_model_base.get_topic(topic_id))

Displaying the baseline topic overview...


,Topic,Count,Name,Representation,Representative_Docs
0,-1,3476,-1_free_can_sleep_they,"[free, can, sleep, they, subscription, pay, no...",[i got this app for a week trial and nowhere c...
1,0,614,0_free_pay_if_anything,"[free, pay, if, anything, can, do, money, paid...","[is not free app 😅, is this app free, it is no..."
2,1,342,1_meditation_meditations_more_find,"[meditation, meditations, more, find, guided, ...","[this app could be great, but it's missing som..."
3,2,274,2_trial_free trial_free_day,"[trial, free trial, free, day, charged, day fr...","[free trial, and then it's really not free., t..."
4,3,248,3_trial_free trial_free_day,"[trial, free trial, free, day, day free, day t...",[you have to make an account to use it. after ...
5,4,223,4_listen_sounds_sound_listen anything,"[listen, sounds, sound, listen anything, music...",[if you don't pay for premium you can't listen...
6,5,209,5_crashes_open_content_phone,"[crashes, open, content, phone, screen, time, ...",[i've been using calm for over 2 years. since ...
7,6,185,6_refund_charged_subscription_email,"[refund, charged, subscription, email, they, a...","[the app is probably ok, customer service suck..."
8,7,177,7_subscription_subscribe_monthly_monthly subsc...,"[subscription, subscribe, monthly, monthly sub...","[requires subscription, requires subscription,..."
9,8,148,8_sounds_sound_soundscapes_audio,"[sounds, sound, soundscapes, audio, noise, fre...",[i had this app and used it all the time. and ...



--- Detailed View of Top 10 Baseline Topics ---

--- Words for Baseline Topic #0 ---
[('free', 0.029343679438044428), ('pay', 0.019430194626553224), ('if', 0.014769411450221379), ('anything', 0.013968874619093458), ('can', 0.013077551992254204), ('do', 0.012164619247279325), ('money', 0.011963613025357799), ('paid', 0.01188215737315636), ('nothing', 0.011860532567415413), ('everything', 0.011575510752806575)]

--- Words for Baseline Topic #1 ---
[('meditation', 0.03891212010874858), ('meditations', 0.023841856381930485), ('more', 0.011011908332489542), ('find', 0.01035180974067678), ('guided', 0.009339899593088856), ('meditate', 0.008744593846801445), ('or', 0.008371553870374486), ('free', 0.0076348288788762745), ('would', 0.007503931144403388), ('what', 0.007378865625329998)]

--- Words for Baseline Topic #2 ---
[('trial', 0.0692893792843331), ('free trial', 0.05247655808340174), ('free', 0.029761010500020845), ('day', 0.026588636081069053), ('charged', 0.019820495125829327), ('day f

In [8]:
# --- PART A: Detailed mapping of Topic IDs to specific names ---

print("Mapping final Topic IDs to names and high-level themes for Baseline App...")
# Assign the raw topic IDs from the model to your baseline DataFrame
df_base['topic_id'] = topics_base
topic_id_to_name_base = {
    0: "Paywall & Lack of Free Content",
    1: "Meditation Content Issues",
    2: "Free Trial Complaints & Charges",
    3: "Redundant: Free Trial Complaints",
    4: "Paywalled Audio & Music",
    5: "Technical: App Crashing / Not Opening",
    6: "Refund & Billing Issues",
    7: "Forced Subscription to Use",
    8: "Soundscape & Audio Feature Issues",
    9: "Subscription Cancellation Problems",
    10: "Misleading 'Free' Label",
    11: "Redundant: Refund & Billing Issues",
    12: "Content Available Free on YouTube",
    13: "False Advertising",
    14: "Generic Negative Feedback ('doesn't work')",
    15: "Redundant: Subscription Cancellation",
    16: "Price is Too Expensive",
    17: "Redundant: Paywall & Lack of Free Content",
    18: "Redundant: Subscription Cancellation",
    19: "Free Version is Useless",
    20: "Issues with Sleep Content",
    21: "Features Locked Behind Paywall",
    22: "Redundant: Issues with Sleep Content",
    23: "Exploiting Anxiety for Money",
    24: "Redundant: Paywalled Features",
    25: "App Uninstalled Due to Cost",
    26: "Generic Positive/Neutral Feedback",
    27: "Redundant: App Uninstalled",
    28: "Redundant: Paywalled Sleep Content",
    29: "Redundant: Paywall & Lack of Free Content",
    30: "Redundant: Paywall & Lack of Free Content",
    31: "Technical Issues with Wearables (Watch)",
    32: "Meta: 1-Star Review Complaints",
    33: "Perceived as a Scam",
    34: "Price is Too Expensive ($70/year)",
    35: "Noise/Junk Topic (mm, diva, bb)",
    36: "Redundant: Paywall & Lack of Free Content",
    37: "Technical Issues with Casting (Google Home)",
    38: "Redundant: Locked/Paid Features",
    39: "Account & Login Issues",
    40: "Noise/Junk Topic (crickets)",
    41: "Issues with Specific Narrator (Tamara Levitt)",
    42: "Redundant: Locked/Paid Features",
    43: "Redundant: Account & Login Issues",
    44: "Redundant: Sound/Audio Issues"
}

df_base['topic_name'] = df_base['topic_id'].map(topic_id_to_name_base)
df_base['topic_name'].fillna("Specific/Niche Complaint", inplace=True)


# --- PART B: Mapping specific names to high-level themes (This remains largely the same) ---
topic_name_to_theme_base = {
    # Theme 1: Monetization & Value
    "Paywall & Lack of Free Content": "Monetization & Value",
    "Free Trial Complaints & Charges": "Monetization & Value",
    "Paywalled Audio & Music": "Monetization & Value",
    "Forced Subscription to Use": "Monetization & Value",
    "Refund & Billing Issues": "Monetization & Value",
    "Misleading 'Free' Label": "Monetization & Value",
    "Subscription Cancellation Problems": "Monetization & Value",
    "Price is Too Expensive": "Monetization & Value",
    "Content Available Free on YouTube": "Monetization & Value",
    "False Advertising": "Monetization & Value",
    "Features Locked Behind Paywall": "Monetization & Value",
    "App Uninstalled Due to Cost": "Monetization & Value",
    "Perceived as a Scam": "Monetization & Value",
    "Exploiting Anxiety for Money": "Monetization & Value",
    "Free Version is Useless": "Monetization & Value",
    "Price is Too Expensive ($70/year)": "Monetization & Value",
    "Redundant: Free Trial Complaints": "Monetization & Value",
    "Redundant: Refund & Billing Issues": "Monetization & Value",
    "Redundant: Subscription Cancellation": "Monetization & Value",
    "Redundant: Paywall & Lack of Free Content": "Monetization & Value",
    "Redundant: Paywalled Sleep Content": "Monetization & Value",
    "Redundant: Locked/Paid Features": "Monetization & Value",

    # Theme 2: Content-Specific Issues
    "Meditation Content Issues": "Content-Specific Issues",
    "Soundscape & Audio Feature Issues": "Content-Specific Issues",
    "Issues with Sleep Content": "Content-Specific Issues",
    "Issues with Specific Narrator (Tamara Levitt)": "Content-Specific Issues",
    "Redundant: Issues with Sleep Content": "Content-Specific Issues",
    "Redundant: Sound/Audio Issues": "Content-Specific Issues",

    # Theme 3: Technical Performance & Bugs
    "Technical: App Crashing / Not Opening": "Technical Performance",
    "Technical Issues with Wearables (Watch)": "Technical Performance",
    "Technical Issues with Casting (Google Home)": "Technical Performance",
    "Account & Login Issues": "Technical Performance",
    "Redundant: Account & Login Issues": "Technical Performance",

    # Theme 4: Other
    "Generic Negative Feedback ('doesn't work')": "Other/Misc.",
    "Generic Positive/Neutral Feedback": "Other/Misc.",
    "Meta: 1-Star Review Complaints": "Other/Misc.",
    "Noise/Junk Topic (mm, diva, bb)": "Other/Misc.",
    "Noise/Junk Topic (crickets)": "Other/Misc.",
    "Specific/Niche Complaint": "Other/Misc."
}

df_base['theme'] = df_base['topic_name'].map(topic_name_to_theme_base)
df_base.loc[df_base['topic_id'] == -1, 'theme'] = 'Outliers / Generic'
df_base['theme'].fillna("Other/Misc.", inplace=True) # Catch any leftovers

print("\n--- Final, Corrected Theme Distribution for Baseline App (Calm) ---")
display(df_base['theme'].value_counts())

Mapping final Topic IDs to names and high-level themes for Baseline App...

--- Final, Corrected Theme Distribution for Baseline App (Calm) ---


theme
Monetization & Value       3491
Outliers / Generic         3476
Content-Specific Issues     730
Other/Misc.                 443
Technical Performance       366
Name: count, dtype: int64

In [9]:
# --- Emotional Analysis: Measuring Sentiment by Theme (Robust Version) ---

# This cell uses a powerful pre-trained Transformer model to get accurate sentiment scores.
print("Setting up Transformer sentiment analysis pipeline for Baseline App...")

# 1. LOAD TOKENIZER AND MODEL
MODEL_NAME = "cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"Using device: {device}")


# 2. TOKENIZE THE DATA
print("Tokenizing all baseline reviews...")
review_list_base = df_base['review_text'].tolist()

inputs_base = tokenizer(
    review_list_base, 
    padding=True, 
    truncation=True, 
    max_length=512,
    return_tensors="pt"
)
inputs_base = {key: val.to(device) for key, val in inputs_base.items()}
print("Tokenization complete.")


# 3. PERFORM INFERENCE IN BATCHES
print("Running model inference in batches...")
all_logits_base = []
batch_size = 32
dataset_base = TensorDataset(inputs_base['input_ids'], inputs_base['attention_mask'])
loader_base = DataLoader(dataset_base, batch_size=batch_size)

with torch.no_grad():
    for batch in tqdm(loader_base, desc="Analyzing Batches"):
        input_ids, attention_mask = batch
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        all_logits_base.append(outputs.logits)

all_logits_base = torch.cat(all_logits_base, dim=0)
probabilities = torch.nn.functional.softmax(all_logits_base, dim=-1)
predictions = torch.argmax(probabilities, dim=-1)
print("Inference complete.")


# 4. PROCESS THE RESULTS
id_to_label = model.config.id2label
predicted_labels = [id_to_label[pred.item()] for pred in predictions]
label_to_score = {'positive': 1, 'neutral': 0, 'negative': -1}
sentiment_scores = [label_to_score[label] for label in predicted_labels]

df_base['sentiment_score'] = sentiment_scores
print("Sentiment scores added to the baseline DataFrame.")

Setting up Transformer sentiment analysis pipeline for Baseline App...


config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

...

Using device: cuda
Tokenizing all baseline reviews...


model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

Tokenization complete.
Running model inference in batches...


Analyzing Batches:   0%|          | 0/266 [00:00<?, ?it/s]

Inference complete.
Sentiment scores added to the baseline DataFrame.


In [10]:
# We'll use the 'df_base' DataFrame that has the 'theme' and 'topic_name' columns.
# Let's ensure our topic names are clean for the charts.
df_base['topic_name'] = df_base['topic_name'].str.replace("Redundant: ", "")

### Deep Dive 1: Why Are Users Upset About Monetization?

In [11]:
# Filter for the theme
monetization_df = df_base[df_base['theme'] == 'Monetization & Value']
# Get the breakdown
monetization_breakdown = monetization_df['topic_name'].value_counts().reset_index()
monetization_breakdown.columns = ['Specific Complaint', 'Review Count']

# Visualize
fig_monetization = px.treemap(
    monetization_breakdown.head(10),
    path=[px.Constant("Monetization Complaints"), 'Specific Complaint'],
    values='Review Count',
    title='<b>Breakdown of Monetization Complaints</b>',
    color_discrete_sequence=px.colors.sequential.Reds_r
)
fig_monetization.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig_monetization.show()

### Deep Dive 2: What Content Are Users Complaining About?

In [1]:
# Filter for the theme
content_df = df_base[df_base['theme'] == 'Content-Specific Issues']
# Get the breakdown
content_breakdown = content_df['topic_name'].value_counts().reset_index()
content_breakdown.columns = ['Specific Complaint', 'Review Count']

# Visualize
fig_content = px.treemap(
    content_breakdown.head(10),
    path=[px.Constant("Content Complaints"), 'Specific Complaint'],
    values='Review Count',
    title='<b>Breakdown of Content-Specific Complaints</b>',
    color_discrete_sequence=px.colors.sequential.Blues_r
)
fig_content.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig_content.show()

NameError: name 'df_base' is not defined

### Deep Dive 3: What Technical Problems Are Users Facing?

In [13]:
# Filter for the theme
technical_df = df_base[df_base['theme'] == 'Technical Performance']
# Get the breakdown
technical_breakdown = technical_df['topic_name'].value_counts().reset_index()
technical_breakdown.columns = ['Specific Complaint', 'Review Count']

# Visualize
fig_technical = px.treemap(
    technical_breakdown.head(10),
    path=[px.Constant("Technical Complaints"), 'Specific Complaint'],
    values='Review Count',
    title='<b>Breakdown of Technical Performance Complaints</b>',
    color_discrete_sequence=px.colors.sequential.Greens_r
)
fig_technical.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig_technical.show()

In [14]:
# In your Kaggle Cell

# --- The Ultimate Breakdown: Sunburst Chart ---
print("Creating a hierarchical Sunburst chart to show theme breakdowns...")

# We need a DataFrame with three columns: Theme, Specific Complaint, and Count
# We can create this by combining our previous breakdown dataframes.
breakdown_df = df_base[~df_base['theme'].isin(['Outliers / Generic', 'Other/Misc.'])]
sunburst_data = breakdown_df.groupby(['theme', 'topic_name']).size().reset_index(name='count')

# Create the Sunburst chart
fig_sunburst = px.sunburst(
    sunburst_data,
    path=['theme', 'topic_name'], # This defines the hierarchy
    values='count',
    title='<b>Anatomy of a Negative Review: A Hierarchical View of Complaints</b>',
    color='theme', # Color the inner ring by theme
    color_discrete_sequence=px.colors.qualitative.Pastel
)

fig_sunburst.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig_sunburst.show()

Creating a hierarchical Sunburst chart to show theme breakdowns...


In [15]:
# In your Kaggle Cell

# --- The "Problem Priority Matrix": Sentiment vs. Frequency ---
print("Creating a Sentiment vs. Frequency scatter plot...")

# 1. Get the Frequency data
theme_freq = df_base['theme'].value_counts().reset_index()
theme_freq.columns = ['Theme', 'Frequency (Number of Reviews)']

# 2. Get the Sentiment data
theme_sent = df_base.groupby('theme')['sentiment_score'].mean().reset_index()
theme_sent.columns = ['Theme', 'Average Sentiment Score']

# 3. Merge them into one DataFrame
priority_df = pd.merge(theme_freq, theme_sent, on='Theme')
priority_df = priority_df[~priority_df['Theme'].isin(['Outliers / Generic', 'Other/Misc.'])]


# 4. Create the Scatter Plot
fig_scatter = px.scatter(
    priority_df,
    x='Frequency (Number of Reviews)',
    y='Average Sentiment Score',
    text='Theme', # Label each point with the theme name
    size='Frequency (Number of Reviews)', # Make bubbles bigger for more frequent topics
    color='Average Sentiment Score',
    color_continuous_scale='Reds_r',
    title='<b>Problem Priority Matrix: Which Complaints Matter Most?</b>',
    template='plotly_white'
)

# Add annotations to create quadrants
fig_scatter.update_traces(textposition='top center')
fig_scatter.add_vline(x=priority_df['Frequency (Number of Reviews)'].mean(), line_dash="dash", annotation_text="Avg. Frequency")
fig_scatter.add_hline(y=priority_df['Average Sentiment Score'].mean(), line_dash="dash", annotation_text="Avg. Sentiment")
fig_scatter.update_layout(title_x=0.5)
fig_scatter.show()

Creating a Sentiment vs. Frequency scatter plot...


# **TIME SERIES ANALYSIS**

In [19]:
# 1. Prepare the data
df_base['date'] = pd.to_datetime(df_base['date'], errors='coerce')
df_base.dropna(subset=['date'], inplace=True)
plot_df_time_base = df_base[~df_base['theme'].isin(['Outliers / Generic', 'Other/Misc.', 'Uncategorized'])]

# 2. Group by month
monthly_trends_base = plot_df_time_base.groupby([pd.Grouper(key='date', freq='M'), 'theme']).size().reset_index(name='review_count')
monthly_trends_base['month'] = monthly_trends_base['date'].dt.to_period('M').dt.to_timestamp()

# 3. Create the Visualization
fig_timeseries_base = px.line(
    monthly_trends_base,
    x='month',
    y='review_count',
    color='theme',
    markers=True,
    title='<b>Baseline App (Calm): Evolution of User Complaints Over Time</b>',
    labels={'month': 'Month', 'review_count': 'Number of Negative Reviews'},
    template='plotly_white'
)

fig_timeseries_base.update_layout(title_x=0.5, legend_title_text='Complaint Theme')
fig_timeseries_base.show()

In [ ]:
# --- Final Step: Save the Enriched DataFrame ---

# Define the output filename
output_filename_base = 'baseline_app_themed_and_scored.csv'

# Save the DataFrame to a new CSV file
df_base.to_csv(output_filename_base, index=False)

print(f"Successfully saved the final, enriched dataset to: {output_filename_base}")
print("You can now find this file in the 'Output' section of this notebook on Kaggle.")